### We will Cover
 - PySpark Dataframe
 - Reading The Dataset
 - Checking the Datatypes of the Column(Schema)
 - Selecting Columns And Indexing
 - Checking Describe opion similar to Pandas
 - Adding Columns
 - Dropping Columns
 - Rename Columns

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('Dataframe_01').getOrCreate()

In [5]:
spark

In [6]:
#Function to get all files in my directory

from os import listdir
from os.path import isfile, join

def get_file_from_directory(folder_dir='', file_name=''):
    filtered_list = []
    file_list = []

    try:
        if folder_dir != '':
            file_list = [f for f in listdir(folder_dir) if isfile(join(folder_dir, f))]

        if file_name != '':
            for arquivo in file_list:
                if file_name in arquivo:
                    filtered_list.append(file_name)
        else:
            return 'Insera o nome do arquivo a ser localizado!'
        
    except FileNotFoundError as error:
        print(f"Insira um caminho valido para a pasta de arquivos! \n Error: {error.filename}")
        
    except IndexError as index_error:
        print(f"Arquivo não localozado {index_error}, certifique-se de inserir o nome do arquivo.")
        
    try:
        return (f"{folder_dir}/{filtered_list[0]}", 'arquivo não localizado')[len(filtered_list) == 0]
    
    except IndexError:
        return 'Arquivo não localizado, talvez o nome do arquivo esteja errado.'    

In [7]:
# set var values
folder='/home/toni/dev_spark/notebooks/files'
csv_file = 'cidades.csv'


In [8]:
print(get_file_from_directory(folder, csv_file))

/home/toni/dev_spark/notebooks/files/cidades.csv


In [11]:
spark.read.csv(get_file_from_directory(folder, csv_file), sep=';', header=True).show()

+----------+--------------+------------+
|cod_cidade|   nome_cidade|sigla_estado|
+----------+--------------+------------+
|         1|        Recife|          PE|
|         2|     São Paulo|          SP|
|         3|  Porto Alegre|          RS|
|         4|   João Pessoa|          PB|
|         5|Rio de Janeiro|          RJ|
+----------+--------------+------------+



In [12]:
# set columns according to the values:>> inferSchema 
df_pyspark = spark.read.csv(get_file_from_directory(folder, csv_file), sep=';', header=True, inferSchema=True)

In [13]:
# check data type of columns
df_pyspark.printSchema()

## df_pyspark.dtypes

## df_pyspark.describe()

## df_pyspark.describe().show()

root
 |-- cod_cidade: integer (nullable = true)
 |-- nome_cidade: string (nullable = true)
 |-- sigla_estado: string (nullable = true)



In [14]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [15]:
# show columns
df_pyspark.columns

['cod_cidade', 'nome_cidade', 'sigla_estado']

In [16]:
# show some rows
df_pyspark.head(2)

[Row(cod_cidade=1, nome_cidade='Recife', sigla_estado='PE'),
 Row(cod_cidade=2, nome_cidade='São Paulo', sigla_estado='SP')]

In [17]:
# Select columns
df_pyspark.select('nome_cidade')

## df_pyspark.select(['nome_cidade','sigla_estado'])
## df_pyspark['nome_cidade']

DataFrame[nome_cidade: string]

In [18]:
# Select columns
df_pyspark.select('nome_cidade').show()

## df_pyspark.select(['nome_cidade','sigla_estado']).show()

+--------------+
|   nome_cidade|
+--------------+
|        Recife|
|     São Paulo|
|  Porto Alegre|
|   João Pessoa|
|Rio de Janeiro|
+--------------+



In [20]:
df_pyspark.describe().show()

+-------+------------------+-----------+------------+
|summary|        cod_cidade|nome_cidade|sigla_estado|
+-------+------------------+-----------+------------+
|  count|                 5|          5|           5|
|   mean|               3.0|       null|        null|
| stddev|1.5811388300841898|       null|        null|
|    min|                 1|João Pessoa|          PB|
|    max|                 5|  São Paulo|          SP|
+-------+------------------+-----------+------------+



In [21]:
# add columns in a df
from pyspark.sql import functions as F

df_pyspark=df_pyspark.withColumn('data_de_hoje',F.current_date())

In [22]:
df_pyspark.select('data_de_hoje').show()

+------------+
|data_de_hoje|
+------------+
|  2021-12-30|
|  2021-12-30|
|  2021-12-30|
|  2021-12-30|
|  2021-12-30|
+------------+



In [23]:
## add columns
df_pyspark=df_pyspark.withColumn('coluna_exemplo',df_pyspark['cod_cidade']+1)

In [24]:
df_pyspark

DataFrame[cod_cidade: int, nome_cidade: string, sigla_estado: string, data_de_hoje: date, coluna_exemplo: int]

In [25]:
## Drop the columns
df_pyspark=df_pyspark.drop('coluna_exemplo')

In [26]:
df_pyspark.show()

+----------+--------------+------------+------------+
|cod_cidade|   nome_cidade|sigla_estado|data_de_hoje|
+----------+--------------+------------+------------+
|         1|        Recife|          PE|  2021-12-30|
|         2|     São Paulo|          SP|  2021-12-30|
|         3|  Porto Alegre|          RS|  2021-12-30|
|         4|   João Pessoa|          PB|  2021-12-30|
|         5|Rio de Janeiro|          RJ|  2021-12-30|
+----------+--------------+------------+------------+



In [28]:
## Rename columns
df_pyspark.withColumnRenamed('data_de_hoje','today').show()

+----------+--------------+------------+----------+
|cod_cidade|   nome_cidade|sigla_estado|     today|
+----------+--------------+------------+----------+
|         1|        Recife|          PE|2021-12-30|
|         2|     São Paulo|          SP|2021-12-30|
|         3|  Porto Alegre|          RS|2021-12-30|
|         4|   João Pessoa|          PB|2021-12-30|
|         5|Rio de Janeiro|          RJ|2021-12-30|
+----------+--------------+------------+----------+

